In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import pyspark as sp

In [2]:
spark = sp.sql.SparkSession.builder.appName("Read Data").master("local[*]").config("spark.driver.memory", "8g").getOrCreate()

In [3]:
df = spark.read.option("header", True).option("inferSchema", True).option("multiLine", False).option("escape", "\"").option("mode", "DROPMALFORMED").csv("D:\MTA_Subway_Hourly_Ridership__2020-2024\MTA_Subway_Hourly_Ridership__2020-2024.csv")

df = df.repartition(64)

In [4]:
df.printSchema()

root
 |-- transit_timestamp: string (nullable = true)
 |-- transit_mode: string (nullable = true)
 |-- station_complex_id: string (nullable = true)
 |-- station_complex: string (nullable = true)
 |-- borough: string (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- fare_class_category: string (nullable = true)
 |-- ridership: integer (nullable = true)
 |-- transfers: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- Georeference: string (nullable = true)



In [5]:
spark.conf.set("spark.hadoop.io.native.lib.available", "false")
spark.conf.set("spark.hadoop.fs.file.impl.disable.cache", "true")
spark.conf.set("spark.hadoop.hadoop.native.io", False)


In [6]:
df.write.mode("overwrite").option("compression", "snappy").partitionBy("transit_mode", "borough").parquet("D:/data/mta_hourly")

In [7]:
spark.stop()